# Exploratory Data Analysis for Wide World Importers

## About this analysis
Wide World Importers is a fake company that sells a variety of different products such as slippers, coffee mugs, and flash drives. The company has extensive records of invoices that have been created during the past several years from several different programs. Here I will explore the data to determine if there is anything noteworthy that executives should incorporate into their business strategy. The data will eventually be run through a machine learning model and thus cannot contain any null values. There are some holes in the data set that need "cleaning".

Cleaning will be performed in the following order,
- Fix standardization errors, where applicable
- Fix business logic errors, where applicable
- Impute null values, where possible
- Drop columns with many null values, where applicable
- Drop rows with null values, where applicable
- Drop duplicate rows, where applicable
- Identify outliers and fix them as instructed


Once cleaning is complete, I will answer:
- Which customer has the most invoices?
- Which item has the highest total quantity sold?
- Which item has generated the most total profit?
- Which item do customers purchase the most quantity of, on average?

In [ ]:
import pandas as pd
df = pd.read_csv('invoices.csv')

## Cleaning
As described above, cleaning will be performed in the following order:
- Fix standardization errors, where applicable
- Fix business logic errors, where applicable
- Impute null values, where possible
- Drop columns with many null values, where applicable
- Drop rows with null values, where applicable
- Drop duplicate rows, where applicable
- Identify outliers and fix them as instructed

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.shape

The customer column has a few standardization errors to address.

In [ ]:
df['Customer'].unique()

In [ ]:
df['Customer'].value_counts()

In [ ]:
df['Customer'] = df['Customer'].replace({"Terry's Trinket" : "Terry's Trinkets"
  , "Stuf by Stew" : "Stuff by Stew"
  , "stuff by stew" : "Stuff by Stew"})

In [ ]:
df['Customer'].value_counts()

There are more null values than not in the LastEditedWhen Column. Because this data is not pertinent to any questions about this analysis, the LastEditedWhen column has been removed.

In [ ]:
df['LastEditedWhen'].isnull().sum()

In [ ]:
df.drop(columns= ['LastEditedWhen'] , inplace = True)

In [ ]:
df.info()

In [ ]:
df['TaxAmount'].isnull().sum()

In [ ]:
df['ExtendedPrice'].isnull().sum()

Now we can see the TaxAmount column has the most null values(120,925), followed by the Extended Price Column.

In [ ]:
df[df['StockItemID'].isnull() & df['Description'].isnull()].shape[0]

In [ ]:
df.info()

In [ ]:
df.dropna(subset = ['StockItemID' , 'Description'] , how='all' , inplace = True)

There are 461 rows where both the StockItemID and Description have null values. Because we intend to determine answers to questions about specific items and their quanitities, rows without this data won't be very helpful so they have been removed.

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df[df['TaxRate'] < 0]

In [ ]:
df[df['StockItemID'] == 47.0]

We can see there are some negative values in the TaxRate column. With a little detective work, we can see the tax rates for each StockItemID are consistent. Based on this, it is determined the negative values were entered in error. To fix this, negative values will be turned into postive rates.

In [ ]:
df.loc[df['TaxRate'] < 0 , 'TaxRate'] *= -1

In [ ]:
df.describe()

In [ ]:
df['TaxRate'].mean().round(2)

With that fixed, it is determined the average TaxRate is 14.98.

Impute:
While TaxAmount has null values, we know this can be determined using the UnitPrice, Quantity, and TaxRate - they have no null values.
UnitPrice, Quantity, and TaxRate are multiplied and then divided by 100 to calculate the TaxAmount and fill null values.

In [ ]:
df['TaxAmount'] = df['TaxAmount'].fillna((df['UnitPrice'] * df['Quantity'] * df['TaxRate']) / 100 )

In [ ]:
df['TaxAmount'].isnull().sum()

In [ ]:
df.describe()

In [ ]:
df.loc[df['TaxAmount'].isna(), 'TaxAmount'] = ( df['UnitPrice'] * df['Quantity'] * df['TaxRate'] / 100 )

In [ ]:
df['TaxAmount'].mean().round(2)

After trying a couple different methods, the Average TaxAmount after removing null values is 112.95.

Next, we will also impute data for the ExtendedPrice column using (UnitPrice x Quantity) + TaxAmount.

In [ ]:
df['ExtendedPrice'] = (df['UnitPrice'] * df['Quantity']) + df['TaxAmount']

In [ ]:
df.describe()

In [ ]:
df['ExtendedPrice'].mean().round(2)

Outliers: Observing the Quantity and ExtendedPrice columns, determining the Z-Score:

In [ ]:
#Quantity Column:

AvgQuan = df['Quantity'].mean()
StdDevQuan = df['Quantity'].std()

In [ ]:
df['Z-ScoreQ'] = (df['Quantity'] - AvgQuan) / StdDevQuan
df.head()

In [ ]:
df.loc[df['Z-ScoreQ'] >= 3]

In [ ]:
#ExtendedPrice Column
AvgExtPrice = df['ExtendedPrice'].mean()
StdDevExtPrice = df['ExtendedPrice'].std()

In [ ]:
df['Z-ScoreExP'] = (df['ExtendedPrice'] - AvgExtPrice) / StdDevExtPrice
df.head()

In [ ]:
df.loc[df['Z-ScoreExP'] >= 3]

In [ ]:
df.info()

Between Quantity and ExtendedPrice, there are between 4 and 6 thousand outliers. While compared to 230,548 it's not a huge amount, it is worth noting.

Next, the data is checked for duplicates in the InvoiceLineID column since these should have unique IDs.

In [ ]:
df[df['InvoiceLineID'].duplicated()]

In [ ]:
df.drop_duplicates(subset = 'InvoiceLineID' , inplace= True)

In [ ]:
df.info()

At this point, the data is clear of most nulls - the only null values being in the Description column.


## Analysis

*Stuff by Stew* has the most unique invoices at 100,628.

In [ ]:
df['Customer'].value_counts()

Looking at the Description and Quantity columns, the *Black and orange fragile despatch tape 48mmx75m* has the highest total quantity sold at 207,108.


In [ ]:
df.groupby('Description')['Quantity'].sum().sort_values(ascending=False).reset_index()

In [ ]:
df.groupby('Description')['LineProfit'].sum().sort_values(ascending=False).reset_index()

The *20 mm Double sided bubble wrap 50m	* has generated the most total profit at 5,283,560.

On average, *Black and orange fragile despatch tape 48mmx75m* is purchased in greatest quantities in a single order; Quantity of 199.53.
This was determined using the Quantity and Description columns.

In [ ]:
itemQuan_per_order = df.groupby(['InvoiceLineID' , 'Description'])['Quantity'].sum().sort_values(ascending=False).reset_index()

In [ ]:
itemQuan_per_order.groupby('Description')['Quantity'].mean().sort_values(ascending=False).round(2).reset_index()

## Results
To summarize, we've answered:
- Which customer has the most invoices? *Stuff by Stew* has 100,628 unique invoices.
- Which item has the highest total quantity sold? *Black and orange fragile despatch tape 48mmx75m* has the highest total quantity sold at 207,108.
- Which item has generated the most total profit? *The *20 mm Double sided bubble wrap 50m* has generated the most total profit at 5,283,560.
- Which item do customers purchase the most quantity of, on average? *Black and orange fragile despatch tape 48mmx75m* with an average quantity of 199.53.